# Code to extract data rainfall values from extracted dates

In [1]:
import pandas as pd
import requests

### 1. Accessing data from data.gov.sg website

In [2]:
rainy_days = pd.read_csv("rainy_days_15mm.csv")
url = "https://api.data.gov.sg/v1/environment/rainfall"

data_df = []

def extract_data():
    for index, row in rainy_days.iterrows():
        date = f'{row["Year"]}-{row["Month"]:02}-{row["Day"]:02}'
        params = {"date": date} # YYYY-MM-DD
        data_dict = requests.get(url, params=params).json()
        readings_lst = data_dict["items"]
        readings_df = pd.DataFrame.from_dict(readings_lst)
        data_df.append(readings_df)
    
data = pd.concat(data_df, ignore_index=True)

## WARNING: takes 75 minutes to extract 1131 days' worth of data
# extract_data()
    

In [4]:
## export extracted data
# data.to_csv("rain_values_15mm.csv")

In [31]:
## view data

data

,timestamp,readings
0,2023-02-03T00:05:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_..."
1,2023-02-03T00:10:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_..."
2,2023-02-03T00:15:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_..."
3,2023-02-03T00:20:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_..."
4,2023-02-03T00:25:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_..."
...,...,...
317617,2020-09-09T23:35:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_..."
317618,2020-09-09T23:40:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_..."
317619,2020-09-09T23:45:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_..."
317620,2020-09-09T23:50:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_..."


#### 2. Convert the 'readings' column into a more useful format

In [33]:
# convert readings col from a list of dicts containing the keys "station_id" and "value" to a dict of station_id-value key-value pairs
def spread_column(lst):
    new_dict = dict()
    for ddict in lst:
        new_dict[ddict["station_id"]] = ddict["value"]
    return new_dict

data["loc_val"] = data["readings"].map(lambda entry: spread_column(entry))

data = data.join(pd.json_normalize(data["loc_val"]))

data

,timestamp,readings,loc_val,S77,S109,S64,S90,S61,S114,S50,...,S24,S82,S11,S122,S91,S07,S29,S92,S46,S105
0,2023-02-03T00:05:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...","{'S77': 0, 'S109': 0, 'S64': 0, 'S90': 0, 'S61...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-03T00:10:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...","{'S77': 0, 'S109': 0, 'S64': 0, 'S90': 0, 'S61...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-03T00:15:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...","{'S77': 0, 'S109': 0, 'S64': 0, 'S90': 0, 'S61...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-03T00:20:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...","{'S77': 0, 'S109': 0, 'S64': 0, 'S90': 0, 'S61...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-03T00:25:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...","{'S77': 0, 'S109': 0, 'S64': 0, 'S90': 0, 'S61...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317617,2020-09-09T23:35:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...","{'S77': 0, 'S109': 0, 'S117': 0, 'S90': 0, 'S6...",0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN
317618,2020-09-09T23:40:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...","{'S77': 0, 'S109': 0, 'S117': 0, 'S90': 0, 'S6...",0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN
317619,2020-09-09T23:45:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...","{'S77': 0, 'S109': 0, 'S117': 0, 'S90': 0, 'S6...",0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN
317620,2020-09-09T23:50:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...","{'S77': 0, 'S109': 0, 'S117': 0, 'S90': 0, 'S6...",0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN


In [13]:
# drop redundant columns
new_df = data.drop(columns = ["readings", "loc_val"])
new_df

,timestamp,S77,S109,S64,S90,S61,S114,S50,S107,S215,...,S24,S82,S11,S122,S91,S07,S29,S92,S46,S105
0,2023-02-03T00:05:00+08:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-02-03T00:10:00+08:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-02-03T00:15:00+08:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-02-03T00:20:00+08:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-02-03T00:25:00+08:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317617,2020-09-09T23:35:00+08:00,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN
317618,2020-09-09T23:40:00+08:00,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN
317619,2020-09-09T23:45:00+08:00,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN
317620,2020-09-09T23:50:00+08:00,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN


In [19]:
# reshape dataframe
melted_df = new_df.melt(id_vars = ["timestamp"], var_name = "station")
melted_df

,timestamp,station,value
0,2023-02-03T00:05:00+08:00,S77,0.0
1,2023-02-03T00:10:00+08:00,S77,0.0
2,2023-02-03T00:15:00+08:00,S77,0.0
3,2023-02-03T00:20:00+08:00,S77,0.0
4,2023-02-03T00:25:00+08:00,S77,0.0
...,...,...,...
25727377,2020-09-09T23:35:00+08:00,S105,NaN
25727378,2020-09-09T23:40:00+08:00,S105,NaN
25727379,2020-09-09T23:45:00+08:00,S105,NaN
25727380,2020-09-09T23:50:00+08:00,S105,NaN


In [22]:
# remove NaNs and 0's from "values" column

final_df = melted_df.dropna()
no_rain_idx = final_df[ (final_df["value"] == 0)].index
final_df.drop(no_rain_idx, inplace=True)
final_df.reset_index(drop = True, inplace = True)

final_df

C:\Users\natas\AppData\Local\Temp\ipykernel_27276\3997747874.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.drop(no_rain_idx, inplace=True)


,timestamp,station,value
0,2023-02-03T10:25:00+08:00,S77,0.2
1,2023-02-03T17:10:00+08:00,S77,0.2
2,2023-02-03T17:15:00+08:00,S77,0.4
3,2023-02-03T17:20:00+08:00,S77,0.2
4,2023-02-03T17:25:00+08:00,S77,0.6
...,...,...,...
679726,2020-01-07T01:55:00+08:00,S105,0.2
679727,2020-01-07T02:05:00+08:00,S105,0.2
679728,2020-01-07T02:10:00+08:00,S105,0.2
679729,2020-01-07T02:20:00+08:00,S105,0.2


In [26]:
# split timestamp column into 3 cols
final_df[['date_time', 'timezone']] = final_df['timestamp'].str.split('+', expand=True)
final_df[['date', 'time']] = final_df['date_time'].str.split('T', expand=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679731 entries, 0 to 679730
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   timestamp  679731 non-null  object 
 1   station    679731 non-null  object 
 2   value      679731 non-null  float64
 3   date_time  679731 non-null  object 
 4   timezone   679731 non-null  object 
 5   date       679731 non-null  object 
 6   time       679731 non-null  object 
dtypes: float64(1), object(6)
memory usage: 36.3+ MB


C:\Users\natas\AppData\Local\Temp\ipykernel_27276\2899607720.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['date_time', 'timezone']] = final_df['timestamp'].str.split('+', expand=True)
C:\Users\natas\AppData\Local\Temp\ipykernel_27276\2899607720.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[['date', 'time']] = final_df['date_time'].str.split('T', expand=True)


,date,time,station,value
0,2023-02-03,10:25:00,S77,0.2
1,2023-02-03,17:10:00,S77,0.2
2,2023-02-03,17:15:00,S77,0.4
3,2023-02-03,17:20:00,S77,0.2
4,2023-02-03,17:25:00,S77,0.6
...,...,...,...,...
679726,2020-01-07,01:55:00,S105,0.2
679727,2020-01-07,02:05:00,S105,0.2
679728,2020-01-07,02:10:00,S105,0.2
679729,2020-01-07,02:20:00,S105,0.2


In [34]:
# extract necessary columns and export

final_df = final_df[["date", "time", "station", "value"]]
# final_df.to_csv("rain_data.csv")